In [1]:
import os
import pandas as pd
import numpy as np
import pickle as pkl

## Matriz de peak de frecuencia v/s periodo objetivo (con múltiplos)

In [2]:
def sort_periodogram(per):
    local_maxima = np.where((per[1:-1] > per[:-2]) & (per[1:-1] > per[2:]))[0] + 1
    idx_highest = local_maxima[np.argsort(per[local_maxima])[::-1]]
    return idx_highest

In [3]:
def top_frequencies(top, idx_highest, freq):
    frequencies = [freq[i] for i in idx_highest]
    return frequencies[:top]

In [4]:
# tamaño max-1 + max-2
def generator_multiples(max):
    mult = np.concatenate([np.arange(1, max, 1), # multiples
                                1./np.arange(2, max, 1)]) # submultiples
    return mult

In [5]:
generator_multiples(5)

array([1.        , 2.        , 3.        , 4.        , 0.5       ,
       0.33333333, 0.25      ])

In [6]:
def relative_error(observed, actual):
    return np.absolute((observed - actual)/actual)

In [7]:
def matrix_band(frequencies_ranking, max, top, period, tol,multiples):
    matrix = [ [ [] for i in range(2*max-3) ] for j in range(top) ]
    # para que acepte solo un tol True
    error_tol_true = []
    for i in range(len(frequencies_ranking)):
        for j in range(len(multiples)):
            error = relative_error(1/(frequencies_ranking[i]*multiples[j]), period)
            if (error < tol):
                if len(error_tol_true) == 0:
                    error_tol_true = [error, i, j]
                    tol_bool = True
                else:
                    if error_tol_true[0] >  error:
                        matrix[error_tol_true[1]][error_tol_true[2]] = [error_tol_true[0], False]
                        error_tol_true = [error, i, j]
                        tol_bool = True
                    else:
                        tol_bool = False
                    
            else:
                tol_bool = False
            matrix[i][j] = [error, tol_bool]
    return matrix

In [9]:
directory = '../dataset'
max_mult = 5
top = 10
tol = 1e-2
multiples = generator_multiples(max_mult)
data_global = pd.read_pickle('../dataset/data_global.pkl')
for folder_type in os.listdir(directory):
    # d_folder_type = 
    d_folder_type = os.path.join(directory, folder_type)
    # folder_type = 'ECL' 'RR'
    if folder_type.find('.') == -1:
        freq = pd.read_pickle('../dataset/'+folder_type+'/frequencies_'+folder_type+'.pkl')
        #lc_folder = '530580438478016768'
        for lc_folder in os.listdir(d_folder_type):
            # porque esta frequencies_ECL.pkl
            if lc_folder.find('.') == -1:
                # d_lc_folder
                dict_matrix = {}
                dict_matrix['multiples'] = multiples
                period = data_global[data_global.index == int(lc_folder)].values[0][0]
                d_lc_folder = os.path.join(d_folder_type,lc_folder)
                d_periodograms = os.path.join(d_lc_folder, 'periodograms.pkl')
                dict_periodograms = pd.read_pickle(d_periodograms)
                for band in dict_periodograms.keys():
                    per = dict_periodograms[band]
                    idx_highest = sort_periodogram(per)
                    freq_ranking = top_frequencies(top, idx_highest,freq)
                    matrix = matrix_band(freq_ranking,max_mult,top,period,tol, multiples)
                    dict_matrix[band] = [freq_ranking, matrix]
                with open(d_lc_folder+'/error_matrixes'+lc_folder+'.pkl', 'wb') as handle:
                    pkl.dump(dict_matrix, handle, protocol=pkl.HIGHEST_PROTOCOL)

        


In [ ]:
def print_matrix(dict_matrix):
    print('Multiples: ', dict_matrix['multiples'])
    

In [ ]:
def print_matrix(matrix, top, max, freq_ranking):
    for i in range(top):
        print(f'freq ranking {i+1}: {freq_ranking[i]:0.4f}', end='  ')
        for j in range(2*max-3):
            
            print(f"[{matrix[i][j][0]:0.4f}, {matrix[i][j][1]}]", end=" ")
        print()

In [ ]:
period = data_global[data_global.index == int(name)].values[0][0]
1/period

1.6197401397919018

name = '246756973652292992' tiene true en 0,0 y 0,5

In [ ]:
name = '246756973652292992'
dict_matrix = pd.read_pickle('../dataset/RR/'+name+'/error_matrixes'+name+'.pkl')
print('Multiples: ', dict_matrix['multiples'])
for band in ['BP', 'G']:
    if band != 'multiples':
        print('Band: '+band)
        matrix = dict_matrix[band][1]
        freq_ranking = dict_matrix[band][0]
        print_matrix(matrix,10,5,freq_ranking)


Multiples:  [1.         2.         3.         4.         0.5        0.33333333
 0.25      ]
Band: BP
freq ranking 1: 2.0437  [0.0000, True] [0.5000, False] [0.6667, False] [0.7500, False] [1.0000, False] [2.0000, False] [3.0000, False] 
freq ranking 2: 2.2540  [0.0933, False] [0.5466, False] [0.6978, False] [0.7733, False] [0.8134, False] [1.7201, False] [2.6268, False] 
freq ranking 3: 1.9068  [0.0718, False] [0.4641, False] [0.6427, False] [0.7320, False] [1.1436, False] [2.2154, False] [3.2872, False] 
freq ranking 4: 9.9413  [0.7944, False] [0.8972, False] [0.9315, False] [0.9486, False] [0.5888, False] [0.3833, False] [0.1777, False] 
freq ranking 5: 2.0650  [0.0103, False] [0.5052, False] [0.6701, False] [0.7526, False] [0.9794, False] [1.9691, False] [2.9588, False] 
freq ranking 6: 2.0546  [0.0053, False] [0.5026, False] [0.6684, False] [0.7513, False] [0.9894, False] [1.9841, False] [2.9788, False] 
freq ranking 7: 1.7242  [0.1853, False] [0.4073, False] [0.6049, False] [0.703

In [ ]:
for band in ['RP', 'multiband']:
    if band != 'multiples':
        print('Band: '+band)
        matrix = dict_matrix[band][1]
        freq_ranking = dict_matrix[band][0]
        print_matrix(matrix,10,5,freq_ranking)

Band: RP
freq ranking 1: 2.0437  [0.0000, True] [0.5000, False] [0.6667, False] [0.7500, False] [1.0000, False] [2.0000, False] [3.0000, False] 
freq ranking 2: 2.2540  [0.0933, False] [0.5466, False] [0.6978, False] [0.7733, False] [0.8134, False] [1.7201, False] [2.6268, False] 
freq ranking 3: 1.9067  [0.0719, False] [0.4641, False] [0.6427, False] [0.7320, False] [1.1437, False] [2.2156, False] [3.2874, False] 
freq ranking 4: 9.9412  [0.7944, False] [0.8972, False] [0.9315, False] [0.9486, False] [0.5888, False] [0.3833, False] [0.1777, False] 
freq ranking 5: 2.0651  [0.0104, False] [0.5052, False] [0.6701, False] [0.7526, False] [0.9793, False] [1.9689, False] [2.9586, False] 
freq ranking 6: 2.0546  [0.0053, False] [0.5026, False] [0.6684, False] [0.7513, False] [0.9894, False] [1.9841, False] [2.9788, False] 
freq ranking 7: 1.7242  [0.1853, False] [0.4073, False] [0.6049, False] [0.7037, False] [1.3706, False] [2.5559, False] [3.7412, False] 
freq ranking 8: 1.5913  [0.2843, 

dict_matrix = {'multiples': [],
    'BP': [freq_ranking, matrix],
    'G': [freq_ranking, matrix],
    'RG': [freq_ranking, matrix],
    'multiband': [freq_ranking, matrix]
}